#### Performance comparison

I use the the Cosmetics Dataset, with many associations. However, we reduce transactions to be maximum 5 in lnegth.
From there, I compare apriori, fp-grwoth and the modifieed fp-grwoth for performance.

Kaggle Dataset: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop <br>
The Dataset is Open-Source <br>
This file contains behavior data for 5 months (Oct 2019 – Feb 2020) from a medium cosmetics online store.<br>

In [61]:
import os
import pandas as pd
import glob
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [62]:
import glob
import os

# setting the path for joining multiple files
files = os.path.join(os.getcwd(), "datasets/eCommerce_purchase_history_from_cosmetics_store/*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
data_raw = pd.concat(map(pd.read_csv, files), ignore_index=True).iloc[:,1:].rename(columns={"user_session":"order_id"})


In [68]:
data_raw["date"] = pd.to_datetime(data_raw['event_time']).dt.date
max_date = data_raw["date"].max()
data_raw["margin"] = 0.1
data_raw["profit"] = data_raw["price"] * data_raw["margin"]
max_profit = data_raw["profit"].max()

data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data = data[data["product_id"].apply(len)<=5]


C:\Users\david\AppData\Local\Temp/ipykernel_16012/4203080727.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))


In [69]:
data_prepared = data["product_id"]

data_prepared

order_id
0000d04d-3516-42ea-bdb9-584e92ce619e             [5809100, 5809101, 5854897, 5888573]
000277a9-30da-4bb2-b5ad-96ef24483af8    [5815567, 5815568, 5815569, 5815570, 5815571]
000289fb-c2fe-4d96-81f2-d44c09249aec                                        [5853035]
0003ece6-f25b-4d8c-92ef-107c6e351e44                      [5818361, 5754853, 5705001]
0004e273-eebe-488c-9189-d838568db29c                      [5686925, 5767492, 5788783]
                                                            ...                      
fffc673e-5513-4621-98d5-5502e5a6ba8b                                        [5814042]
fffcc9cb-2b36-4e96-a07a-970188a42491                      [5827368, 5796757, 5668432]
fffd7c77-e85c-47a3-b2a7-e132752660bf                               [5815035, 5823657]
ffff78e3-1e8f-4eda-bb42-95749ab4f42e    [5686083, 5710585, 5776129, 5776130, 5776132]
ffff8da3-b79a-48f2-888c-117f2d1a7793                               [5816170, 5899849]
Name: product_id, Length: 77264, dtype: objec

In [70]:
te = TransactionEncoder()
te_ary = te.fit(data_prepared).transform(data_prepared)
T = pd.DataFrame(te_ary, columns=te.columns_)
T

,3762,3763,3774,3776,3806,3928,3929,3936,3945,3959,...,5931955,5931982,5931991,5932153,5932154,5932518,5932520,5932521,5932578,5932583
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77259,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
77260,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
77261,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
77262,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [71]:
frequent_itemsets = apriori(T, min_support=0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets


,support,itemsets,length
0,0.002304,(3762),1
1,0.001268,(3978),1
2,0.003559,(4185),1
3,0.002369,(4497),1
4,0.001618,(4938),1
...,...,...,...
383,0.001074,"(5814516, 5814518)",2
384,0.001087,"(5814517, 5814518)",2
385,0.001592,"(5823667, 5819894)",2
386,0.001061,"(5809912, 5809910, 5809911)",3


In [74]:
frequent_itemsets = fpgrowth(T, min_support=0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.006678,(5854897),1
1,0.001281,(5853035),1
2,0.002097,(5754853),1
3,0.003326,(5686925),1
4,0.003145,(5649270),1
...,...,...,...
383,0.001087,"(5814517, 5814518)",2
384,0.001074,"(5814516, 5814518)",2
385,0.001048,"(5814516, 5814517, 5814518)",3
386,0.002071,"(5677043, 5697463)",2


In [72]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.0001)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(4185),(3762),0.003559,0.002304,0.001424,0.400000,173.626966,0.001415,1.662827
1,(3762),(4185),0.002304,0.003559,0.001424,0.617978,173.626966,0.001415,2.608330
2,(5528034),(5528035),0.003520,0.008050,0.001437,0.408088,50.692169,0.001408,1.675840
3,(5528035),(5528034),0.008050,0.003520,0.001437,0.178457,50.692169,0.001408,1.212936
4,(5584836),(5584838),0.002110,0.001721,0.001048,0.496933,288.684164,0.001045,1.984383
5,(5584838),(5584836),0.001721,0.002110,0.001048,0.609023,288.684164,0.001045,2.552296
6,(5584836),(5584844),0.002110,0.001838,0.001010,0.478528,260.372937,0.001006,1.914123
7,(5584844),(5584836),0.001838,0.002110,0.001010,0.549296,260.372937,0.001006,2.214069
8,(5759491),(5651975),0.005915,0.004349,0.001126,0.190372,43.776493,0.001100,1.229764
9,(5651975),(5759491),0.004349,0.005915,0.001126,0.258929,43.776493,0.001100,1.341416


In [73]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"


rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.0001,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,consequent,sup_consequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
0,[],NA,[5560754],200,[5560754],200,0.002589,NA,NA,NA,38782.880000,0.020267,0.000000,0.000000,38782.880000,0.000000
1,[],NA,[89343],51,[89343],51,0.000660,NA,NA,NA,15518.120000,0.008109,0.000000,0.000000,15518.120000,0.000000
2,[],NA,[5877454],344,[5877454],344,0.004452,NA,NA,NA,15155.510000,0.007920,0.000000,0.000000,15155.510000,0.000000
3,[],NA,[5850281],102,[5850281],102,0.001320,NA,NA,NA,14239.470000,0.007441,0.000000,0.000000,14239.470000,0.000000
4,[],NA,[5846437],292,[5846437],292,0.003779,NA,NA,NA,13313.550000,0.006957,0.000000,0.000000,13313.550000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,"[5831586, 5831969]",6,[5831585],21,"[5831586, 5831969, 5831585]",4,0.000052,0.666667,2452.825397,0.666395,191.597843,0.000100,128.336765,63.261078,106.040000,-42.778922
2563,[5463],143,[5560758],33,"[5560758, 5463]",1,0.000013,0.006993,16.37296,0.006566,191.589231,0.000100,694.480000,-502.890769,186.732727,-689.623497
2564,[5759492],628,[5708350],52,"[5708350, 5759492]",7,0.000091,0.011146,16.561979,0.010473,191.568341,0.000100,1480.620000,-1289.051659,175.064615,-1464.116274
2565,[5864475],95,[5864464],30,"[5864475, 5864464]",3,0.000039,0.031579,81.330526,0.031191,191.430000,0.000100,1507.650000,-1316.220000,143.820000,-1460.040000
